In [1]:
!pip install catboost

     |████████████████████████████████| 65.7MB 47kB/s 


In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission_QrCyCoT.csv to sample_submission_QrCyCoT.csv
Saving test_YCcRUnU.csv to test_YCcRUnU.csv
Saving train_Df64byy.csv to train_Df64byy.csv


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

In [4]:
train_df = pd.read_csv('/content/train_Df64byy.csv')
test_df = pd.read_csv('/content/test_YCcRUnU.csv')

In [67]:
def make_df_ready(df):
  df = df.set_index('ID')
  df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].replace('14+', '15.0').astype(float)
  df['Is_Spouse'] = df['Is_Spouse'].map({'No': 1, 'Yes': 0})
  df['Accomodation_Type'] = df['Accomodation_Type'].map({'Owned': 1, 'Rented': 0})
  df['Reco_Insurance_Type'] = df['Reco_Insurance_Type'].map({'Individual': 1, 'Joint': 0})
  df['Holding_Policy_Type'] = df['Holding_Policy_Type'].astype(str)
  df['Reco_Policy_Cat'] = df['Reco_Policy_Cat'].astype(str)
  df['Region_Code'] = df['Region_Code'].astype(str)
  df['Health Indicator'] = df['Health Indicator'].astype(str)
  return df

In [8]:
train_df = train_df.set_index('ID')

In [11]:
train_df['Holding_Policy_Duration'] = train_df['Holding_Policy_Duration'].replace('14+', '15.0').astype(float)

In [15]:
train_df['Is_Spouse'] = train_df['Is_Spouse'].map({'No': 1, 'Yes': 0})

In [17]:
train_df['Accomodation_Type'] = train_df['Accomodation_Type'].map({'Owned': 1, 'Rented': 0})
train_df['Reco_Insurance_Type'] = train_df['Reco_Insurance_Type'].map({'Individual': 1, 'Joint': 0})

In [47]:
train_df['Holding_Policy_Type'] = train_df['Holding_Policy_Type'].astype(str)
train_df['Reco_Policy_Cat'] = train_df['Reco_Policy_Cat'].astype(str)
train_df['Region_Code'] = train_df['Region_Code'].astype(str)
train_df['Health Indicator'] = train_df['Health Indicator'].astype(str)

In [59]:
X = train_df.loc[:, [i for i in train_df.columns if i != 'Response']]
y = train_df.loc[:, 'Response']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [61]:
clf = CatBoostClassifier(iterations=200, task_type='GPU')

In [62]:
clf.fit(X=X_train, y=y_train, cat_features=['City_Code', 'Region_Code', 'Health Indicator', 'Holding_Policy_Type', 'Reco_Policy_Cat'])

Learning rate set to 0.116284
0:	learn: 0.6379462	total: 37ms	remaining: 7.35s
1:	learn: 0.6006049	total: 69.1ms	remaining: 6.84s
2:	learn: 0.5699593	total: 101ms	remaining: 6.64s
3:	learn: 0.5462714	total: 135ms	remaining: 6.62s
4:	learn: 0.5277840	total: 167ms	remaining: 6.51s
5:	learn: 0.5139361	total: 199ms	remaining: 6.43s
6:	learn: 0.5034413	total: 230ms	remaining: 6.33s
7:	learn: 0.4968997	total: 265ms	remaining: 6.37s
8:	learn: 0.4906273	total: 296ms	remaining: 6.28s
9:	learn: 0.4856473	total: 324ms	remaining: 6.16s
10:	learn: 0.4820563	total: 352ms	remaining: 6.04s
11:	learn: 0.4776281	total: 379ms	remaining: 5.93s
12:	learn: 0.4750582	total: 405ms	remaining: 5.82s
13:	learn: 0.4712415	total: 431ms	remaining: 5.72s
14:	learn: 0.4689738	total: 458ms	remaining: 5.64s
15:	learn: 0.4669851	total: 490ms	remaining: 5.63s
16:	learn: 0.4655615	total: 517ms	remaining: 5.56s
17:	learn: 0.4637181	total: 540ms	remaining: 5.46s
18:	learn: 0.4625712	total: 564ms	remaining: 5.37s
19:	learn: 

In [63]:
preds_test = clf.predict(X_test)

In [64]:
print(classification_report(y_test.values, preds_test))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      7668
           1       0.57      0.30      0.39      2509

    accuracy                           0.77     10177
   macro avg       0.68      0.61      0.63     10177
weighted avg       0.74      0.77      0.74     10177



In [65]:
roc_auc_score(y_test.values, preds_test)

0.6122944878874237

In [71]:
test_to_predict = make_df_ready(test_df)

In [79]:
submission_preds = clf.predict(test_to_predict)

In [76]:
sample_df = pd.read_csv('/content/sample_submission_QrCyCoT.csv')

In [80]:
sample_df['Response'] = submission_preds

In [87]:
train_df['Response'].value_counts(normalize=True)

0    0.760053
1    0.239947
Name: Response, dtype: float64

In [90]:
sample_df.to_csv('my_submission.csv', index=False)